In [8]:
import requests
import json

# Función para obtener la región del Pokémon basado en su primera generación
def get_region(gen):
    regions = {
        1: 'KANTO',
        2: 'JOHTO',
        3: 'HOENN',
        4: 'SINNOH',
        5: 'TESELIA',
        6: 'KALOS',
        7: 'ALOLA',
        8: 'GALAR',
        9: 'PALDEA',  # Región añadida
    }
    return regions.get(gen, 'UNKNOWN')

legendary_pokemon_set = {
    144, 145, 146, 150, 151, 243, 244, 245, 249, 250, 251,
    377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 480,
    481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491,
    492, 493, 494, 638, 639, 640, 641, 642, 643, 644, 645,
    646, 647, 648, 649, 716, 717, 718, 772, 773, 785, 786,
    787, 788, 791, 792, 793, 794, 795, 796, 797, 798, 799,
    800, 801, 802, 803, 804, 805, 806, 807, 888, 889, 890,
    891, 892, 893, 894, 895, 896, 897, 898, 1007, 1008, 1009,
    1010, 1011, 1012
}

# Función para comprobar si un número está en el conjunto
def is_legendary(pokedex_number):
    return pokedex_number in legendary_pokemon_set

# Abrir o crear el archivo para escribir los INSERT
with open("02-pokemones.sql", "w") as f:
    # Recorrer los Pokémon de 1 a 1025
    insert_query = f"INSERT INTO pokemones (id, name, descripcion, type1, type2, region, legendary) VALUES \n"
    f.write(insert_query)
    r=1
    for pokemon_id in range(1, 1026):
        # Obtener los datos del Pokémon desde la API
        response = requests.get(f'https://pokeapi.co/api/v2/pokemon/{pokemon_id}')
        if response.status_code == 200:
            data = response.json()

            # Obtener el nombre y otros datos básicos
            name = data['name'].capitalize()
            height = data['height'] / 10  # Altura en metros
            weight = data['weight'] / 10  # Peso en kilogramos
            pokedex_number = pokemon_id  # El número de Pokédex es el ID del Pokémon

            # Crear la descripción a partir de estos datos
            description = f"{name} is a Pokemon with a height of {height}m, a weight of {weight}kg, and its Pokedex number is {pokedex_number}."

            # Tipos del Pokémon
            types = data['types']
            type1 = types[0]['type']['name'].upper()
            type2 = types[1]['type']['name'].upper() if len(types) > 1 else 'NULL'

            # Obtener la región a partir de la primera generación donde aparece
            # region = get_region(1)  # Para este ejemplo, siempre será 'KANTO' por la generación 1
            if (pokemon_id == 152 or pokemon_id == 252 or pokemon_id == 387 or pokemon_id == 495 or pokemon_id == 650 or pokemon_id == 722 or pokemon_id == 810 or pokemon_id == 906):
                r += 1

            region = get_region(r)
            

            # Verificar si es un Pokémon legendario
            legendary = is_legendary(pokemon_id)
            # if 'legendary' in data['species'] and data['species']['legendary']:
            #     legendary = 'true'

            # Escribir la consulta INSERT
            insert_query = f" ({pokemon_id}, '{name}', '{description}', "
            insert_query += f"(SELECT id FROM types WHERE name = '{type1}'), "
            insert_query += f"(SELECT id FROM types WHERE name = '{type2}'), "
            if (pokemon_id!=1025):
                insert_query += f"(SELECT id FROM regions WHERE name = '{region}'), {legendary}),\n"
            else:
                insert_query += f"(SELECT id FROM regions WHERE name = '{region}'), {legendary});"

            f.write(insert_query)

print("Archivo 'pokemons.data' generado con éxito.")



Archivo 'pokemons.data' generado con éxito.
